# Amazon Bedrock Batch Inference

## 情報
### 公開情報
- 開発者ドキュメント: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference.html
- Quota: https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- コードサンプル: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-example.html

### 2024.1.25
- Public Preview
- 利用方法
    - REST API: あると思うが面倒
    - CLI: 無さそう
    - SDK: プレビューのがある
        - PythonとJavaのみ
        - https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
    - コンソール: 無さそう

## 検証内容
- Bedrockのbatch推論のquotaがon demandと関係ない（だからquotaに引っ掛からなくてうれしい）と言う説
- text to image (SDXL or Titan or Both)で500枚くらいバッチ推論ジョブを発行してどんな風に実行が完了するか（もしくは完了しないか）を見てみる
- On demandのquotaより明らかに早かったら嬉しい

## 検証

### データ準備
参考: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-data.html

### 環境準備

In [1]:
! wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip

--2024-01-25 00:47:13--  https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
Resolving d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)... 99.84.216.107, 99.84.216.166, 99.84.216.85, ...
Connecting to d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)|99.84.216.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314169135 (300M) [application/zip]
Saving to: ‘bedrock-python-sdk-reinvent.zip’

bedrock-python-sdk- 100%[===================>] 299.61M  35.6MB/s    in 7.3s    

2024-01-25 00:47:21 (40.8 MB/s) - ‘bedrock-python-sdk-reinvent.zip’ saved [314169135/314169135]



In [18]:
%env SDK = sdk
! unzip -f -d $SDK bedrock-python-sdk-reinvent.zip

env: SDK=sdk
Archive:  bedrock-python-sdk-reinvent.zip


下記より、以下を参照: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-example.html

In [20]:
! pip install $SDK/botocore-1.32.4-py3-none-any.whl
! pip install $SDK/boto3-1.29.4-py3-none-any.whl

Processing ./sdk/botocore-1.32.4-py3-none-any.whl
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.64
    Uninstalling botocore-1.31.64:
      Successfully uninstalled botocore-1.31.64
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.32.4 which is incompatible.
boto3 1.28.64 requires botocore<1.32.0,>=1.31.64, but you have botocore 1.32.4 which is incompatible.
Processing ./sdk/boto3-1.29.4-py3-none-any.whl
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.64
    Uninstalling boto3-1.28.64:
      Successfully uninstalled boto3-1.28.64


In [21]:
import boto3
bedrock = boto3.client(service_name="bedrock")

In [22]:
inputDataConfig = ({
    "s3InputDataConfig": {
        "s3Uri": "s3://input-bucket/input/abc.jsonl"
    }
})

outputDataConfig=({
    "s3OutputDataConfig": {
        "s3Uri": "s3://output-bucket/output/"
    }
})

response = bedrock.create_model_invocation_job(
    roleArn = "arn:aws:iam::123456789012:role/MyBatchInferenceRole",
    modelId = "amazon.titan-text-express-v1",
    jobName = "my-batch-job",
    inputDataConfig = inputDataConfig,
    outputDataConfig = outputDataConfig
)

jobArn = response.get('jobArn')
jobArn

AccessDeniedException: An error occurred (AccessDeniedException) when calling the CreateModelInvocationJob operation: Cross-account pass role is not allowed.